In [8]:
from dataclasses import dataclass
import subprocess
from subprocess import CompletedProcess

from google.cloud import resourcemanager_v3
from google.cloud.resourcemanager_v3.services.projects.client import ProjectsClient

@dataclass
class Config:
    PROJECT_NAME: str = "{{cookiecutter.project_name}}"  # gh repo name
    GITHUB_PAT: str = "{{cookiecutter.github_pat}}"
    GITHUB_AUTHOR: str = "{{cookiecutter.author_github_handle}}"
    GITHUB_CLOUD_BUILD_INSTALLATION_ID: str = (
        "{{cookiecutter.github_cloud_build_app_installation_id}}"
    )
    GCP_PROJECT_ID: str = (
        "{{cookiecutter.gcp_project_id}}"  # name of organizing project in GCP
    )
    GOOGLE_APPLICATION_CREDENTIALS: str = (
        "{{cookiecutter.google_application_credentials}}"
    )
    GCP_REGION_ID: str = "{{cookiecutter.gcp_region_id}}"
    GCP_ARTIFACT_REGISTRY_REPO: str = "{{cookiecutter.gcp_artifact_registry_repo_name}}"
    GCP_TRIGGER_NAME: str = "{{cookiecutter.gcp_trigger_name}}"
    GCP_TRIGGER_PATTERN: str = "{{cookiecutter.trigger_branch_pattern}}"

    run_validation: bool = True

    @property
    def gcp_project_number(self) -> str:
        projects_client: ProjectsClient = resourcemanager_v3.ProjectsClient()
        project_name = f"projects/{self.GCP_PROJECT_ID}"
        project: resourcemanager_v3.Project = projects_client.get_project(
            name=project_name
        )
        project_number = project.name.split("/")[-1]
        return project_number

    @property
    def cloud_build_service_agent_email(self) -> str:
        return f"service-${self.gcp_project_number}@gcp-sa-cloudbuild.iam.gserviceaccount.com"

    @property
    def gcp_github_connection_name(self) -> str:
        return f"{self.PROJECT_NAME}-gh-connection"

    @property
    def gcp_pat_secret_name(self) -> str:
        return f"{self.PROJECT_NAME}-github-pat"

    @property
    def gcp_pat_secret_path(self) -> str:
        return make_secret_path(
            secret_name=self.gcp_pat_secret_name, project_id=self.GCP_PROJECT_ID
        )

    @property
    def github_uri(self) -> str:
        return f"https://github.com/{self.GITHUB_AUTHOR}/{self.PROJECT_NAME}.git"

    ## Validation steps
    def __post_init__(self) -> None:
        if self.run_validation:
            check_github_pat(self.GITHUB_AUTHOR, self.GITHUB_PAT)
            _ = check_github_repo(
                self.GITHUB_AUTHOR, self.PROJECT_NAME, self.GITHUB_PAT
            )


def run_subprocess_w_check(cmds: list[str]) -> CompletedProcess[str]:
    result: CompletedProcess[str] = subprocess.run(
        cmds, check=True, capture_output=True, text=True
    )
    if result.returncode != 0:
        raise RuntimeError(
            f"Command failed with return code {result.returncode}: {result.stderr}"
        )

    return result


def assign_permissions_to_default_cloud_builder_service_account(
    gcp_project_id: str,
    gcp_project_number: str,
) -> CompletedProcess[str]:
    default_cloud_build_service_account = (
        f"service-{gcp_project_number}@gcp-sa-cloudbuild.iam.gserviceaccount.com"
    )

    cmds = [
        "gcloud",
        "projects",
        "add-iam-policy-binding",
        gcp_project_id,
        f"--member=serviceAccount:{default_cloud_build_service_account}",
        "--role=roles/cloudbuild.builds.builder",
    ]

    result = run_subprocess_w_check(cmds)
    return result


In [ ]:
c = Config(GCP_PROJECT_ID='wck-source', run_validation=False)
project_id, project_number = c.GCP_PROJECT_ID, c.gcp_project_number

default_cloud_build_service_account = (
    f"service-{project_number}@gcp-sa-cloudbuild.iam.gserviceaccount.com"
)


('wck-source', '912138393635')

In [ ]:
result = assign_permissions_to_default_cloud_builder_service_account(
    project_id, project_number
)

In [ ]:
%%bash 

gcloud projects add-iam-policy-binding wck-source \
    --member=servi
        "projects",
        "add-iam-policy-binding",
        gcp_project_id,
        f"--member=serviceAccount:{default_cloud_build_service_account}",
        "--role=roles/cloudbuild.builds.builder",

